<center>
<h1>Investigating the Py-ART data model for radar data</h1>
<i>Scott Collis<sup>1</sup>, 
Jonathan Helmus<sup>2</sup>, and
Steve Nesbitt<sup>3</sup>
<br>
<sup>1</sup>Argonne National Laboratory
<sup>2</sup>Anaconda, Inc.
<sup>3</sup>University of Illinois at Urbana-Champaign
<br>

</center>

This notebook explores the data model used for pointing radar data in the Python ARM Radar Toolkit (Py-ART). We will do this by loading a radar file from CF-Radial from ARM's C-Band system in the Southern Great Plains site.

## Getting the data for this exercise

---
Run this cell once to download the data:

In [ ]:
%%script bash
mkdir ../data
wget -P ../data https://www.atmos.illinois.edu/~snesbitt/data/cfrad.20110523_221616.000_to_20110523_222150.1000_sgpc_v0_SUR.nc

## How to install pyart

---
Installing pyart using `conda`:
    
``bash
conda install -c jjhelmus trmm_rsl
conda install -c conda-forge arm_pyart
``

Start your notebooks - now you're good to go!


---
## Let's start the exercise

In [16]:
%matplotlib inline

import pyart

filename = '../data/cfrad.20110523_221616.000_to_20110523_222150.1000_sgpc_v0_SUR.nc'

Read the CF-Radial file into Py-ART's data model for pointing gated data

In [15]:
radar = pyart.io.read_cfradial(filename)

Lets investigate what is at the top level with a dir() command 

In [11]:
dir(radar)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_sweep_in_range',
 '_dic_info',
 'add_field',
 'add_field_like',
 'altitude',
 'altitude_agl',
 'antenna_transition',
 'azimuth',
 'check_field_exists',
 'drift',
 'elevation',
 'extract_sweeps',
 'fields',
 'fixed_angle',
 'gate_altitude',
 'gate_latitude',
 'gate_longitude',
 'gate_x',
 'gate_y',
 'gate_z',
 'georefs_applied',
 'get_azimuth',
 'get_elevation',
 'get_end',
 'get_field',
 'get_gate_x_y_z',
 'get_nyquist_vel',
 'get_slice',
 'get_start',
 'get_start_end',
 'heading',
 'info',
 'init_gate_altitude',
 'init_gate_longitude_latitude',
 'init_gate_x_y_z',
 'init_rays_per_sweep',
 'instrument_parameters',
 'iter_azimuth',
 'iter_elevation',
 'iter_end',
 'iter_field',
 

Anything in the data model which contains array-like data is a dictionary with metadata and the actual data contained in the 'data' key, for example the array which contains information about the elevation angle of the sensor. 

In [ ]:
radar.azimuth.keys()

In [ ]:
radar.azimuth['long_name']

In [ ]:
radar.azimuth['data']

In [ ]:
f = plt.figure(figsize=[15,8])
plt.plot(radar.time['data'], radar.azimuth['data'] )
plt.xlabel(radar.time['long_name'] + ' (' + radar.time['units'] + ')')
plt.ylabel(radar.azimuth['long_name'] + ' (' + radar.azimuth['units'] + ')')

So all the pointing data is contained in the base object, the azimuth and elevation of the antenna/sensor plus the range and time axes

In [ ]:
print radar.range['data'].min(), radar.range['data'].max(), radar.range['units']
f = plt.figure(figsize=[15,8])
plt.plot(radar.time['data'], radar.elevation['data'] )
plt.xlabel(radar.time['long_name'] + ' (' + radar.time['units'] + ')')
plt.ylabel(radar.elevation['long_name'] + ' (' + radar.elevation['units'] + ')')

there is also a swag of metadata contained within, well.. the metadata dictionary

In [ ]:
for mykey in radar.metadata.keys():
    print mykey, ': ', radar.metadata[mykey]

Now the final top level bit of information, the data model we use follows CF-Radial morphology and hence has a set of "helper" fields to format out the radar coverage pattern. That is, to seperate sweeps.

In [ ]:
radar.sweep_end_ray_index['data']
f = plt.figure(figsize=[15,8])
for i in range(len(radar.sweep_end_ray_index['data'])):
    start_index = radar.sweep_start_ray_index['data'][i]
    end_index = radar.sweep_end_ray_index['data'][i]
    plt.plot(radar.time['data'][start_index:end_index], 
             radar.elevation['data'][start_index:end_index], 
             label = 'Sweep number '+ str(radar.sweep_number['data'][i]))
plt.legend()
plt.xlabel(radar.time['long_name'] + ' (' + radar.time['units'] + ')')
plt.ylabel(radar.elevation['long_name'] + ' (' + radar.elevation['units'] + ')')    

In [ ]:
import numpy as np
print np.unique(radar.elevation['data'])

Now to the actual data, or what ARM would call Primary Measurements. This is all stored in the field field of the radar object and is a dictionary of dictionaries. Best shown by example:

In [ ]:
print radar.fields.keys()
print ""
for mykey in radar.fields.keys():
    print mykey,':', radar.fields[mykey]['long_name'] + ' (' + radar.fields[mykey]['units'] + ')'

As far as CF-Radial ingest and write is concerned the variable names correspond to the variable names, the non-array data to the variable attributes and the 'data' key to the array.. lets look at some data

In [ ]:
f = plt.figure(figsize=[15,8])
my_pc = plt.pcolormesh(radar.range['data'], radar.time['data'],
                       radar.fields['reflectivity_horizontal']['data'])
plt.xlabel(radar.range['long_name'] + ' (' + radar.range['units'] + ')')
plt.ylabel(radar.time['long_name'] + ' (' + radar.time['units'] + ')')
cb = plt.colorbar(mappable = my_pc)
cb.set_label(radar.fields['reflectivity_horizontal']['standard_name'] +\
             ' (' + radar.fields['reflectivity_horizontal']['units'] + ')')
    


And of course we can use our sweep indicators to isolate a single sweep

In [ ]:
f = plt.figure(figsize=[15,8])
start_index = radar.sweep_start_ray_index['data'][0]
end_index = radar.sweep_end_ray_index['data'][0]
my_pc = plt.pcolormesh(radar.range['data'], radar.time['data'][start_index:end_index],
                       radar.fields['reflectivity_horizontal']['data'][start_index:end_index, :])
plt.xlabel(radar.range['long_name'] + ' (' + radar.range['units'] + ')')
plt.ylabel(radar.time['long_name'] + ' (' + radar.time['units'] + ')')
cb = plt.colorbar(mappable = my_pc)
cb.set_label(radar.fields['reflectivity_horizontal']['standard_name'] +\
             ' (' + radar.fields['reflectivity_horizontal']['units'] + ')')

We can get a quick overview of what is contained in the radar object using the **info** method.

In [ ]:
radar.info('compact')   # see what happens with 'standard' or 'full'

This functionality is also available from the command line using the **radar_info** command.

In [ ]:
!radar_info --compact data/cfrad.20110523_220916.000_to_20110523_221451.000_sgpcsapr_v0_SUR.nc

Thus concludes the intro! Py-ART, of course, can do all this for you including pretty PPIs etc.. but this gives an introduction to the data model we use.  Questions? Comments? Science Lead: <a href = 'mailto:scollis@anl.gov'> Scott Collis</a>.